# Vizualizace fraktálů

## Textový popis

Tento projekt se zaměřuje na vizualizaci fraktálů. Cílem je
implementovat algoritmy pro efektivní generování známých fraktálů, jako
jsou Mandelbrotova a Juliova množina (pro
$f\left(z\right)=z^{2}+c,c\in\mathbb{C}$), a vytvořit interaktivní
vizualizace (viz [Matplotlib
widgets](https://matplotlib.org/stable/gallery/widgets/index.html))
těchto fraktálů pomocí knihovny Matplotlib. Možností je také využít knihovnu [Pygame](https://www.pygame.org/news) pro vytvoření interaktivních vizualizací.

Výstupem projektu budou interaktivní vizualizace fraktálů, které
umožňují uživateli prozkoumávat různé části fraktálu a přizpůsobovat
parametry pro generování fraktálů ($c\in\mathbb{C}$ pro Juliovu
množinu).

## Funkcionality

- Implementovat algoritmus pro efektivní generování Mandelbrotovy
  množiny pomocí knihoven NumPy a Numba
- Implementovat algoritmus pro efektivní generování Juliovy množiny
  (pro $f\left(z\right)=z^{2}+c,c\in\mathbb{C}$) pomocí knihoven NumPy a Numba
- Vytvořit funkci pro vizualizaci fraktálů pomocí knihovny Matplotlib,
  která zobrazuje fraktály pomocí barevného mapování podle iterací
  potřebných k dosažení určitého prahu
- Implementovat interaktivní prvky vizualizace, které umožňují
  uživateli (v případě implementace skrze Pygame lze nahradit ovladáním klávesnicí a myší):
  - přiblížit nebo oddálit fraktál
  - měnit barevné schéma vykreslení počtu iterací do divergence
  - přizpůsobovat parametry pro generování fraktálů (např. počet
    iterací, $c$)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba
from ipywidgets import interact, IntSlider, ToggleButtons, FloatSlider

# Interaktivní implementace Mandelbrotovy množiny

In [2]:
@numba.njit(parallel=True)
def mandelbrot_set(x_min=-2.0, x_max=1.0, y_min=-1.5, y_max=1.5, n=500, k=100, x=0, y=0, zoom=1.0):
    scale = 1.0 / zoom
    x_min *= scale
    x_max *= scale
    y_min *= scale
    y_max *= scale
    re = np.linspace(x_min+x, x_max+x, n)
    im = np.linspace(y_min+y, y_max+y, n)
    
    divergence_matrix = np.full((n, n), k, dtype=np.int32)
    
    # Prochazeni vsech bodu v rovine
    for ix in numba.prange(n):
        for iy in range(n):
            c = re[ix] + 1j * im[iy]
            z = 0.0 + 0.0j
            
            # Iterace pro vypocet Mandelbrotovy posloupnosti
            for i in range(k):
                z = z * z + c
                
                # Kontrola divergence, pokud |z| > 2 (bez pouziti numpy, nebot to zpomaluje celou funcki)
                if (z.real * z.real + z.imag * z.imag) > 4.0:
                    divergence_matrix[ix, iy] = i
                    break
                    
    return divergence_matrix


In [3]:
n = 2500
k = 500

In [4]:
%timeit mandelbrot_set(n=n, k=k)

342 ms ± 3.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Něco o funkčnosti
Manderbrotova množina obshahuje interktivní slidery pro parametry:
- Rozlišení (počet pixelů stran)
- Počet iterací
- Posuvník po realné ose (x)
- Posuvník po imaginarní ose (y)
- Příblížení na střed
- A dále výběr z pár barevných map (pomocí tlačítek)

Resetovat parametry se dá pomocí opětovného spuštění buňky pod tímhle textem



In [5]:
def plot_mandelbrot(n=500, k=100, cmap='hot', x=0, y=0, zoom=1.0):
    data = mandelbrot_set(n=n, k=k, x=x, y=y, zoom=zoom)
    plt.figure(figsize=(9, 8))
    plt.imshow(data.T, cmap=cmap, origin='lower', extent=[-2.0, 1.0, -1.5, 1.5])
    plt.colorbar(label='Počet iterací do divergence')
    plt.title(f'Mandelbrotova množina (k={k}, n={n})')
    plt.show()

# Interaktivni slidery, tlacitka
interact(plot_mandelbrot,
         n=IntSlider(min=50, max=2500, step=50, value=500, description='Rozlišení'),
         k=IntSlider(min=5, max=500, step=5, value=100, description='Iterace'),
         cmap=ToggleButtons(options=['hot', 'viridis', 'inferno', 'gray', 'turbo'], value='hot', description='Barevná mapa'),
         x=FloatSlider(min=-2.0, max=2.0, step=0.01, value=0.0, description='Posun v x'),
         y=FloatSlider(min=-2.0, max=2.0, step=0.01, value=0.0, description='Posun v y'),
         zoom=FloatSlider(min=1.0, max=50.0, step=0.5, value=1.0, description='Priblížení'));

interactive(children=(IntSlider(value=500, description='Rozlišení', max=2500, min=50, step=50), IntSlider(valu…

# Interaktivni implementace Juliovy mnoziny

In [6]:
@numba.njit(parallel=True)
def julia_set(c_real=-0.7, c_imag=0.27, x_min=-2.0, x_max=2.0, y_min=-2.0, y_max=2.0, n=1000, k=200, x=0.0, y=0.0, zoom=1.0):
    
    scale = 1.0 / zoom
    x_min *= scale
    x_max *= scale
    y_min *= scale
    y_max *= scale
    re = np.linspace(x_min + x, x_max + x, n)
    im = np.linspace(y_min + y, y_max + y, n)
    
    divergence_matrix = np.full((n, n), k, dtype=np.int32)
    
    c = complex(c_real, c_imag)
    
    # Velice podobna s Mandelbrotovou mnozinou, akorat je c pevne nastavene
    for ix in numba.prange(n):
        for iy in range(n):
            z = re[ix] + 1j * im[iy]
            
            for i in range(k):
                z = z * z + c
                if (z.real * z.real + z.imag * z.imag) > 4.0:
                    divergence_matrix[ix, iy] = i
                    break
                    
    return divergence_matrix

In [7]:
%timeit julia_set(n=2000, k=500)

49.8 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Něco o funkčnosti
Juliova množina obshahuje interktivní slidery pro parametry:
- Rozlišení (počet pixelů stran)
- Počet iterací
- Velikost komplexního čísla c
- Posuvník po realné ose (x)
- Posuvník po imaginarní ose (y)
- Příblížení na střed
- A dále výběr z pár barevných map (pomocí tlačítek)

Resetovat parametry se dá pomocí opětovného spuštění buňky pod tímhle textem

In [8]:
def plot_julia(n=1000, k=200, c_real=-0.7, c_imag=0.27015, cmap='hot', x=0, y=0, zoom=1.0):
    data = julia_set(n=n, k=k, c_real=c_real, c_imag=c_imag, x=x, y=y, zoom=zoom)
    plt.figure(figsize=(9, 8))
    plt.imshow(data.T, cmap=cmap, origin='lower', extent=[-2.0, 2.0, -2.0, 2.0])
    plt.colorbar(label='Počet iterací do divergence')
    plt.title(f'Juliova množina (k={k}, n={n})')
    plt.show()

# Interaktivni slidery, tlacitka
interact(plot_julia,
         n=IntSlider(min=50, max=2500, step=50, value=1000, description='Rozlišení'),
         k=IntSlider(min=5, max=500, step=5, value=200, description='Iterace'),
         c_real=FloatSlider(min=-2.5, max=2.5, step=0.01, value=-0.7, description='Realná část c'),
         c_imag=FloatSlider(min=-2.5, max=2.5, step=0.01, value=0.27, description='Imag. část c'),
         cmap=ToggleButtons(options=['hot', 'viridis', 'inferno', 'gray', 'turbo'], value='hot', description='Barevná mapa'),
         x=FloatSlider(min=-2.0, max=2.0, step=0.01, value=0.0, description='Posun v x'),
         y=FloatSlider(min=-2.0, max=2.0, step=0.01, value=0.0, description='Posun v y'),
         zoom=FloatSlider(min=1.0, max=50.0, step=0.5, value=1.0, description='Priblížení'));
         

interactive(children=(IntSlider(value=1000, description='Rozlišení', max=2500, min=50, step=50), IntSlider(val…